In [2]:
pip install python-dotenv

You should consider upgrading via the '/home/clovis1er/Documents/code/deblay/token_liquidity_analysis/notebooks/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# System imports
from dotenv import load_dotenv
import os
from pprint import pprint

In [4]:
# CMC imports
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

In [32]:
# Python imports
import json
from datetime import date, timedelta
import pandas as pd
from datetime import datetime, date

In [6]:
load_dotenv()
CMC_API = os.getenv("CMC_API")

In [7]:
symbols = {
    "BTC": "bitcoin",
    "UNI": "uniswap",
    "XMR": "monero",
    "VET": "vechain", 
    "USDT": "tether", 
    "BNB": "binancecoin",
    "ETH": "ethereum",
    "MATIC": "matic-network",
    "DOT": "polkadot", 
    "DOGE": "dogecoin", 
    "XRP": "ripple", 
    "SAND": "the-sandbox", 
    "SUSHI": "sushi"
}

### Trying to get market data with Coinmarket cap :
###### Note : after this trial, I realized that with the free API, it is not possible to access historical data

In [8]:
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/quotes/historical'
parameters = {
    "symbol": "BTC",
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': CMC_API,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  pprint(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  pprint(e)

{'data': {'BTC': {'id': 4676,
                  'is_active': 3734,
                  'is_fiat': None,
                  'name': '7yguh3s16mp',
                  'quotes': [{'quote': {'USD': {'circulating_supply': 6452,
                                                'market_cap': 0.8851285630429551,
                                                'price': 0.2210689776067396,
                                                'timestamp': '2022-10-08T01:10:56.367Z',
                                                'total_supply': 4386,
                                                'volume_24h': 3184}},
                              'timestamp': '2022-10-08T01:10:56.367Z'},
                             {'quote': {'USD': {'circulating_supply': 8274,
                                                'market_cap': 0.19010901594583762,
                                                'price': 0.4164484441199474,
                                                'timestamp': '2022-10-08T01:10:56.367

### Trying with Coingecko API :

In [84]:
def format_date(d):
    return d.strftime("%d-%m-%Y")

url = "https://api.coingecko.com/api/v3/coins/<coin_id>/market_chart?vs_currency=usd&days=<nb_days>"
date_start = date(2022, 1, 1)
today = date.today()
dfs = []

for tiker, id_ in symbols.items():
    full_url = url.replace("<coin_id>", id_).replace("<nb_days>", str((today-date_start).days))
    res = requests.get(full_url)
    if res.status_code == 200:
        volumes = res.json()["total_volumes"]
        new_df = pd.DataFrame(volumes, columns=["Timestamp", tiker])
        new_df[tiker] = new_df[tiker] /1000000
        dfs.append(new_df)
    else:
        print(f"Error with {tiker} : {res.text}")
   

In [68]:
print(dfs)

[         Timestamp           BTC
0    1641081600000  18904.585811
1    1641168000000  33756.128242
2    1641254400000  21890.019532
3    1641340800000  24895.990584
4    1641427200000  31837.641246
..             ...           ...
276  1664928000000  30974.520028
277  1665014400000  29366.342083
278  1665100800000  31354.758329
279  1665187200000  25445.042398
280  1665191438000  24285.549961

[281 rows x 2 columns],          Timestamp         UNI
0    1641081600000  193.649960
1    1641168000000  200.986078
2    1641254400000  349.235836
3    1641340800000  367.529749
4    1641427200000  598.856112
..             ...         ...
276  1664928000000  166.658973
277  1665014400000  106.455820
278  1665100800000  144.315495
279  1665187200000  136.193574
280  1665191380000  125.651818

[281 rows x 2 columns],          Timestamp         XMR
0    1641081600000  246.723262
1    1641168000000  171.457202
2    1641254400000  159.317411
3    1641340800000  145.903590
4    1641427200000  146.03

In [91]:
crypto_dfs = dfs.copy()
def merge_df(df1, df2):
    return df1.merge(df2, on="Timestamp", how="inner")

def ts_to_date(df):
    return format_date(datetime.fromtimestamp(int(df["Timestamp"])/1000))

for i in range(0, len(crypto_dfs)):
    crypto_dfs[i] = crypto_dfs[i].drop(crypto_dfs[i].tail(1).index)
    crypto_dfs[i]["Timestamp"] = crypto_dfs[i].apply(ts_to_date, axis=1)
    if i == 0:
        df = crypto_dfs[i]
    else:
        df = merge_df(df, crypto_dfs[i])
df = df.set_index(['Timestamp'])
print(df)

                     BTC         UNI         XMR         VET          USDT  \
Timestamp                                                                    
02-01-2022  18904.585811  193.649960  246.723262  185.705631  38101.900235   
03-01-2022  33756.128242  200.986078  171.457202  203.457724  35584.063770   
04-01-2022  21890.019532  349.235836  159.317411  271.100955  49571.178475   
05-01-2022  24895.990584  367.529749  145.903590  332.618442  52215.823894   
06-01-2022  31837.641246  598.856112  146.031430  730.545253  66142.943859   
...                  ...         ...         ...         ...           ...   
04-10-2022  26634.227152  161.864707   68.655632   51.300049  32437.902531   
05-10-2022  30974.520028  166.658973   72.712743   61.132453  35921.416945   
06-10-2022  29366.342083  106.455820   67.070471   48.466786  34389.944867   
07-10-2022  31354.758329  144.315495  108.435413   46.014354  36750.092117   
08-10-2022  25445.042398  136.193574  105.954786   51.066423  31

In [92]:
df.to_csv('volumes.csv')

In [93]:
df.mean(axis=0)

BTC      28114.206401
UNI        179.820542
XMR        144.866936
VET        225.287170
USDT     53543.455393
BNB       1338.531280
ETH      16138.746543
MATIC      732.313729
DOT        635.135677
DOGE       722.586092
XRP       3742.508230
SAND       454.046010
SUSHI      132.281694
dtype: float64